Tout d'abord, nous importons les bibliothèques permettant de récupérer les données. La bliothèque requests permet d'envoyer des requêtes à des sites web, bs4 permet d'analyser et extraire des données de documents HTML et donc de sites internet, re permet de rechercher des expressions régulières (regex) dans du texte, pandas permet de manipuler des bases de données et rapidfuzz fournit des outils pour comparer des chaînes de caractères.

In [3]:
import requests
import bs4
import re
import pandas as pd
from rapidfuzz import fuzz
import s3fs


On cherche tout d'abord à confectionner une table avec les informations sur le livre et le numéro d'index auquel il correspond. Cette base sera ensuite donnée à l'API pour qu'elle donne un ou plusieurs thème à chaque texte. 
La cellule ci-dessous permet de récupérer le texte de la page "gutindex.all", qui associe à chaque oouvrage un numéro d'index. 

In [ ]:
# URL du fichier d'index des textes
url_liste_textes = "https://www.gutenberg.org/dirs/GUTINDEX.ALL.iso-8859-1.txt"

# Téléchargement du fichier d'index
request_liste_textes = requests.get(url_liste_textes).content
page = bs4.BeautifulSoup(request_liste_textes, "lxml")
body = page.find("body")
index_texte = body.get_text()

Toujours dans l'optique de créer une table avec les informations sur le texte dans une colonne et l'index dans l'autre, on ne souhaite garder que le texte correspondant aux informations et à l'index. 
On utilise pour ce faire les balises de début et de fin de l'index puis on supprime les quelques lignes inutiles qui donnent des informations sur le contenu de l'index après avoir converti le texte en une liste de lignes pour faciliter le traitement.

In [ ]:
# Chercher les indices des marqueurs "<===LISTINGS===>" et "<==End of GUTINDEX.ALL==>"
start_marker = "<===LISTINGS===>"
end_marker = "<==End of GUTINDEX.ALL==>"
start_index = index_texte.find(start_marker)
end_index = index_texte.find(end_marker)

# Extraire le texte entre les marqueurs
texte_extrait = index_texte[start_index + len(start_marker):end_index].strip()
texte_extrait_lignes = texte_extrait.splitlines()

# Filtrer les lignes pertinentes
texte_extrait_lignes_trie = texte_extrait_lignes[10:len(texte_extrait_lignes)-1]
texte_complet = '\n'.join(texte_extrait_lignes_trie)

Il nous faut donc maintenant séparer le texte en deux parties, l'une contenant les informations sur l'oeuvre (titre, auteur, date, langue de l'oeuvre...) et l'autre le numéro d'index.
La cellule ci-dessous, après avoir divisé le texte en oeuvres, créé une liste d'oeuvre avec description (contenant les informations sur l'oeuvre) et index séparés. La première utilisation du regex dans la boucle permet d'extraire l'index tandis que la deuxième permet de retirer l'index et les espaces superflus pour ne garder que le texte pour la colonne "Description". On convertit enfin la liste en dataframe pour la manipuler plus efficacement et on ne garde que les textes en français.

In [ ]:
# Diviser le texte en oeuvres
oeuvres = re.split(r'(?=\n{2,})', texte_complet.strip())

# Extraire les descriptions et indices
# Liste pour stocker les données extraites
data = []
for oeuvre in oeuvres:
    # Trouver l'index dans l'oeuvre
    match_index = re.search(r'(?<=\s\s)([\d]+?[A-Z]?)(?=\n)', oeuvre)
    index = match_index.group(1) if match_index else None

    # Nettoyer le texte de l'oeuvre
    description = re.sub(r'(?<=\s\s)([\d]+?[A-Z]?)(?=\n)', '', oeuvre).strip()

    # Ajouter les données
    data.append({"Description": description, "Index": index})

# Convertir les données en DataFrame
df_livres = pd.DataFrame(data)
df_livres_fr = df_livres[df_livres["Description"].str.contains(r"\[Language: French\]", na=False)]


Afin que l'API nous donne bien des thèmes pour les textes envoyés, nous avons fait le choix, après quellques essais, de nous restreindre à ceux d'auteurs célèbres. Nous avons donc réalisé une liste d'auteurs célèbres des 17
La cellule ci-dessous ne garde que les ouvrages dont l'auteur est dans la liste. 

In [5]:
# List of French Writers abritrarily defined and chosen in the 17th, 18th and 19th century
auteurs = [
    # 17th century
    "Honoré d'Urfé", "Madeleine de Scudéry", "Paul Scarron", "Jean de La Fontaine",
    "Madame de Lafayette", "Charles Sorel", "Tristan L'Hermite", "François de Salignac de La Mothe-Fénelon",
    "Savinien de Cyrano de Bergerac",
    
    # 18th century
    "Montesquieu", "Voltaire", "Jean-Jacques Rousseau", "Denis Diderot", "Marivaux",
    "Abbé Prévost", "Pierre Choderlos de Laclos", "Beaumarchais", 
    
    # 19th century
    "Honoré de Balzac", "Victor Hugo", "Alexandre Dumas", "Gustave Flaubert", "Émile Zola",
    "Stendhal", "Alfred de Musset", "George Sand", "Jules Verne", "Alphonse Daudet",
    "Théophile Gautier", "Edmond de Goncourt",
    "Joris-Karl Huysmans", "Octave Mirbeau", 
    "Prosper Mérimée", "Eugène Sue", "Charles Nodier",
    "Gaston Leroux", "François-René de Chateaubriand", "Anatole France", "Gustave Flaubert", "Alfred Jarry",
    "Guy de Maupassant", "Romain Rolland", "Alfred Séguin", "Alfred de Vigny", "Paul de Kock"

]

#On créé une expression réulière que signifie "ou" pour l'utiliser ensuite
auteurs_join = "|".join(map(re.escape, auteurs))
# Filtrer les lignes qui contiennent au moins un des auteurs
df_livres_fr_filtré = df_livres_fr[df_livres_fr["Description"].str.contains(auteurs_join, na=False)]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

MY_BUCKET = "arnaudbrrt"
fs.ls(MY_BUCKET) 
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/Data_libroguessr/livres_fr_triés.csv"

with fs.open(FILE_PATH_OUT_S3, "w") as file_out:
    df_livres_fr_filtré.to_csv(file_out, index =False)

NameError: name 'df_livres_fr' is not defined